In [1]:
import numpy as np
import scipy
import pandas as pd
import math
import random
import sklearn
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%matplotlib inline

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
raw_df = pd.read_csv('cat_de_3year.csv', encoding = "utf-8")#, nrows = 100000)
#raw_df = pd.read_csv('cat_de_15-18.csv', encoding = "utf-8")
#raw_df.head(20)

In [4]:
print('min date is {}'.format(min(raw_df['TDT_T_KEY_ORDER_DATE'])))
print('max date is {}'.format(max(raw_df['TDT_T_KEY_ORDER_DATE'])))

min date is 01-APR-15
max date is 31-OCT-17


# Let us create a data frame that contains only required data for future basic recommender system
### As of now we are not taking into account session information
> Logic for defining the strength of interaction of the user with the item would be
1. Bought an item $(0,2]$ times - weight will be "1.0"
2. Bought an item $(2,5]$ times - weight will be "2.0"
3. Bought an item $(5, +\infty)$ times - weight will be "3.0"
> I can think of a functional weight for the counts of times the item was boubht. TODO

### Required columns are
* CUS_N_KEY_CUSTOMER == personId
* PPR_N_PHY_PRODUCT_ID = contentId
* eventStrength = depends on the overall history of the customer-item interactions. _no basket or date informaiton is used_


In [5]:
## to start let us take a sample of our data
sample_df = raw_df.sample(10000000)
sample_df.head()

,SCM_N_ORDER_POS_NR,SCM_N_ORDER_NR,SCM_N_ORDER_NR_NC,TDT_T_KEY_ORDER_DATE,ART_N_KEY_ARTICLE,PPR_N_KEY_PHYS_PRODUCT,CUS_N_KEY_CUSTOMER,SCM_D_QUANTITY,PPR_N_PHY_ARTICLE_ID,PPR_N_PHY_PRODUCT_ID
23386206,326958669,77010162,0,04-NOV-16,1504045,28918640,8946703,0.3333,46664,35623
42709782,441736935,103571342,0,30-JAN-18,2175175,129039356,14779116,1.0000,74762,52803
40259970,433811738,101687694,0,02-JAN-18,1704074,93213399,773754,1.0000,67355,48482
9794739,215867477,51379636,0,30-MAR-15,24107,16429,3154345,1.0000,16417,12743
25820043,340012140,79932336,0,28-DEC-16,1950041,15411,577695,1.0000,24371,18497


In [6]:
# remove non-used columns
sample_df.drop(['SCM_N_ORDER_POS_NR','SCM_N_ORDER_NR_NC', 'SCM_N_ORDER_NR_NC', 'PPR_N_KEY_PHYS_PRODUCT', 'SCM_N_ORDER_NR', 'ART_N_KEY_ARTICLE','PPR_N_PHY_ARTICLE_ID'], axis=1, inplace=True)
sample_df.head()

,TDT_T_KEY_ORDER_DATE,CUS_N_KEY_CUSTOMER,SCM_D_QUANTITY,PPR_N_PHY_PRODUCT_ID
23386206,04-NOV-16,8946703,0.3333,35623
42709782,30-JAN-18,14779116,1.0000,52803
40259970,02-JAN-18,773754,1.0000,48482
9794739,30-MAR-15,3154345,1.0000,12743
25820043,28-DEC-16,577695,1.0000,18497


In [7]:
# this is not needed since we are simply counting prod id
# sample_more_1_interaction = sample_df.groupby(['CUS_N_KEY_CUSTOMER','PPR_N_PHY_PRODUCT_ID']).filter(lambda x: x['SCM_D_QUANTITY'].unique().size > 1 )

In [8]:
#sample_more_1_interaction.head()

In [9]:
tmp = sample_df.groupby(['CUS_N_KEY_CUSTOMER', 'PPR_N_PHY_PRODUCT_ID']).count()
tmp.reset_index(level=tmp.index.names, inplace=True)
interactions_df = tmp.rename(index=str, columns = {'CUS_N_KEY_CUSTOMER': 'personId', 
                                                   'PPR_N_PHY_PRODUCT_ID': 'contentId', 
                                                   'TDT_T_KEY_ORDER_DATE': 'date', 
                                                   'SCM_D_QUANTITY': 'eventStrength'}).drop('date', axis=1)
display(interactions_df)

,personId,contentId,eventStrength
0,22,15896,2
1,22,17576,3
2,22,19223,1
3,22,19544,1
4,22,19774,1
5,22,19997,2
6,22,23508,1
7,22,25635,3
8,22,25968,1
9,22,26038,1


In [10]:
# Visualize pairplot of df
# sns.pairplot(interactions_df, hue='eventStrength');

Recommender systems have a problem known as user cold-start, in which is hard do provide personalized recommendations for users with none or a very few number of consumed items, due to the lack of information to model their preferences.
For this reason, we are keeping in the dataset only users with at leas 5 interactions.

In [11]:
users_interactions_count_df = interactions_df.groupby(['personId', 'contentId']).size().groupby('personId').size()
print('# users: %d' % len(users_interactions_count_df))
users_with_enough_interactions_df = users_interactions_count_df[users_interactions_count_df >= 5].reset_index()[['personId']]
print('# users with at least 5 interactions: %d' % len(users_with_enough_interactions_df))

# users: 904865
# users with at least 5 interactions: 329797


In [12]:
print('# of interactions: %d' % len(interactions_df))
interactions_from_selected_users_df = interactions_df.merge(users_with_enough_interactions_df, 
               how = 'right',
               left_on = 'personId',
               right_on = 'personId')
print('# of interactions from users with at least 5 interactions: %d' % len(interactions_from_selected_users_df))

# of interactions: 5853869
# of interactions from users with at least 5 interactions: 4815167


In [13]:
####
# This must be changed
###
interactions_full_df = interactions_from_selected_users_df

## Evaluation
Evaluation is important for machine learning projects, because it allows to compare objectivelly different algorithms and hyperparameter choices for models.

One key aspect of evaluation is to ensure that the trained model generalizes for data it was not trained on, using *Cross-validation techniques*. We are using here a simple cross-validation approach named *holdout*, in which a random data sample (20% in this case) are kept aside in the training process, and exclusively used for evaluation. All evaluation metrics reported here are computed using the *test set*.

_Ps. A more robust evaluation approach could be to split train and test sets by a reference date, where the train set is composed by all interactions before that date, and the test set are interactions after that date. For the sake of simplicity, we chose the first random approach for this notebook, but you may want to try the second approach to better simulate how the recsys would perform in production predicting "future" users interactions._

In [14]:
interactions_train_df, interactions_test_df = train_test_split(interactions_full_df,
                                   stratify=interactions_full_df['personId'], 
                                   test_size=0.20,
                                   random_state=42)

print('# interactions on Train set: %d' % len(interactions_train_df))
print('# interactions on Test set: %d' % len(interactions_test_df))

# interactions on Train set: 3852133
# interactions on Test set: 963034


In Recommender Systems, there are a set metrics commonly used for evaluation. We chose to work with *Top-N* accuracy metrics, which evaluates the accuracy of the top recommendations provided to a user, comparing to the items the user has actually interacted in test set.

This evaluation method works as follows:

* For each user
    * For each item the user has interacted in test set
        * Sample 100 other items the user has never interacted. Ps. Here we naively assume those non interacted items are not relevant to the user, which might not be true, as the user may simply not be aware of those not interacted items. But let's keep this assumption.
        * Ask the recommender model to produce a ranked list of recommended items, from a set composed one interacted item and the 100 non-interacted ("non-relevant!) items
        * Compute the Top-N accuracy metrics for this user and interacted item from the recommendations ranked list
* Aggregate the global Top-N accuracy metrics

The Top-N accuracy metric choosen was *Recall@N* which evaluates whether the interacted item is among the top N items (hit) in the ranked list of 101 recommendations for a user.
Ps. Other popular ranking metrics are *NDCG@N* and *MAP@N*, whose score calculation takes into account the position of the relevant item in the ranked list (max. value if relevant item is in the first position). You can find a reference to implement this metrics in this post.

In [15]:
#Indexing by personId to speed up the searches during evaluation
interactions_full_indexed_df = interactions_full_df.set_index('personId')
interactions_train_indexed_df = interactions_train_df.set_index('personId')
interactions_test_indexed_df = interactions_test_df.set_index('personId')

In [33]:
def get_items_interacted(person_id, interactions_df):
    # Get the user's data and merge in the movie information.
    interacted_items = interactions_df.loc[person_id]['contentId']
    return set(interacted_items if type(interacted_items) == pd.Series else [interacted_items])

In [34]:
#Top-N accuracy metrics consts
EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS = 100

class ModelEvaluator:


    def get_not_interacted_items_sample(self, person_id, sample_size, seed=42):
        interacted_items = get_items_interacted(person_id, interactions_full_indexed_df)
        all_items = set(articles_df['contentId'])
        non_interacted_items = all_items - interacted_items

        random.seed(seed)
        non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
        return set(non_interacted_items_sample)

    def _verify_hit_top_n(self, item_id, recommended_items, topn):        
            try:
                index = next(i for i, c in enumerate(recommended_items) if c == item_id)
            except:
                index = -1
            hit = int(index in range(0, topn))
            return hit, index

    def evaluate_model_for_user(self, model, person_id):
        #Getting the items in test set
        interacted_values_testset = interactions_test_indexed_df.loc[person_id]
        if type(interacted_values_testset['contentId']) == pd.Series:
            person_interacted_items_testset = set(interacted_values_testset['contentId'])
        else:
            person_interacted_items_testset = set([int(interacted_values_testset['contentId'])])  
        interacted_items_count_testset = len(person_interacted_items_testset) 

        #Getting a ranked recommendation list from a model for a given user
        person_recs_df = model.recommend_items(person_id, 
                                               items_to_ignore=get_items_interacted(person_id, 
                                                                                    interactions_train_indexed_df), 
                                               topn=10000000000)

        hits_at_5_count = 0
        hits_at_10_count = 0
        #For each item the user has interacted in test set
        for item_id in person_interacted_items_testset:
            #Getting a random sample (100) items the user has not interacted 
            #(to represent items that are assumed to be no relevant to the user)
            non_interacted_items_sample = self.get_not_interacted_items_sample(person_id, 
                                                                          sample_size=EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS, 
                                                                          seed=item_id%(2**32))

            #Combining the current interacted item with the 100 random items
            items_to_filter_recs = non_interacted_items_sample.union(set([item_id]))

            #Filtering only recommendations that are either the interacted item or from a random sample of 100 non-interacted items
            valid_recs_df = person_recs_df[person_recs_df['contentId'].isin(items_to_filter_recs)]                    
            valid_recs = valid_recs_df['contentId'].values
            #Verifying if the current interacted item is among the Top-N recommended items
            hit_at_5, index_at_5 = self._verify_hit_top_n(item_id, valid_recs, 5)
            hits_at_5_count += hit_at_5
            hit_at_10, index_at_10 = self._verify_hit_top_n(item_id, valid_recs, 10)
            hits_at_10_count += hit_at_10

        #Recall is the rate of the interacted items that are ranked among the Top-N recommended items, 
        #when mixed with a set of non-relevant items
        recall_at_5 = hits_at_5_count / float(interacted_items_count_testset)
        recall_at_10 = hits_at_10_count / float(interacted_items_count_testset)

        person_metrics = {'hits@5_count':hits_at_5_count, 
                          'hits@10_count':hits_at_10_count, 
                          'interacted_count': interacted_items_count_testset,
                          'recall@5': recall_at_5,
                          'recall@10': recall_at_10}
        return person_metrics

    def evaluate_model(self, model):
        #print('Running evaluation for users')
        people_metrics = []
        for idx, person_id in enumerate(list(interactions_test_indexed_df.index.unique().values)):
            #if idx % 100 == 0 and idx > 0:
            #    print('%d users processed' % idx)
            person_metrics = self.evaluate_model_for_user(model, person_id)  
            person_metrics['_person_id'] = person_id
            people_metrics.append(person_metrics)
        print('%d users processed' % idx)

        detailed_results_df = pd.DataFrame(people_metrics) \
                            .sort_values('interacted_count', ascending=False)
        
        global_recall_at_5 = detailed_results_df['hits@5_count'].sum() / float(detailed_results_df['interacted_count'].sum())
        global_recall_at_10 = detailed_results_df['hits@10_count'].sum() / float(detailed_results_df['interacted_count'].sum())
        
        global_metrics = {'modelName': model.get_model_name(),
                          'recall@5': global_recall_at_5,
                          'recall@10': global_recall_at_10}    
        return global_metrics, detailed_results_df
    
model_evaluator = ModelEvaluator()  

## Popularity model

A common (and usually hard-to-beat) baseline approach is the Popularity model. This model is not actually personalized - it simply recommends to a user the most popular items that the user has not previously consumed. As the popularity accounts for the "wisdom of the crowds", it usually provides good recommendations, generally interesting for most people.
Ps. The main objective of a recommender system is to leverage the long-tail items to the users with very specific interests, which goes far beyond this simple technique.

In [16]:
#Computes the most popular items
item_popularity_df = interactions_full_df.groupby('contentId')['eventStrength'].sum().sort_values(ascending=False).reset_index()
item_popularity_df.head(10)

,contentId,eventStrength
0,3187,128516
1,38891,115094
2,38333,112624
3,39043,102060
4,39751,99399
5,10178,91057
6,44561,87116
7,42948,84087
8,46365,75847
9,23103,74426


In [19]:
class PopularityRecommender:
    
    MODEL_NAME = 'Popularity'
    
    def __init__(self, popularity_df, items_df=None):
        self.popularity_df = popularity_df
        self.items_df = items_df
        
    def get_model_name(self):
        return self.MODEL_NAME
        
    def recommend_items(self, user_id, items_to_ignore=[], topn=10, verbose=False):
        # Recommend the more popular items that the user hasn't seen yet.
        recommendations_df = self.popularity_df[~self.popularity_df['contentId'].isin(items_to_ignore)] \
                               .sort_values('eventStrength', ascending = False) \
                               .head(topn)

        if verbose:
            if self.items_df is None:
                raise Exception('"items_df" is required in verbose mode')

            recommendations_df = recommendations_df.merge(self.items_df, how = 'left', 
                                                          left_on = 'contentId', 
                                                          right_on = 'contentId')[['eventStrength', 'contentId', 'title', 'url', 'lang']]


        return recommendations_df
    
# sample_df
# popularity_model = PopularityRecommender(item_popularity_df, articles_df)
articles_df = sample_df.rename(index=str, columns = {'CUS_N_KEY_CUSTOMER': 'personId', 
                                                   'PPR_N_PHY_PRODUCT_ID': 'contentId'})
popularity_model = PopularityRecommender(item_popularity_df, articles_df)

Here we perform the evaluation of the Popularity model, according to the method described above.
It achieved the Recall@5 of ????, which means that about ??% of interacted items in test set were ranked by Popularity model among the top-5 items (from lists with 100 random items). And Recall@10 was even higher (???%), as expected.
It might be surprising to you that usually Popularity models could perform so well!

In [20]:
print('Evaluating Popularity recommendation model...')
pop_global_metrics, pop_detailed_results_df = model_evaluator.evaluate_model(popularity_model)
print('\nGlobal metrics:\n%s' % pop_global_metrics)
pop_detailed_results_df.head(10)

Evaluating Popularity recommendation model...


KeyboardInterrupt: 

#### Content-Based Filtering model is NOT applicable to the data we are working on with right now
Content-based filtering approaches leverage description or attributes from items the user has interacted to recommend similar items. It depends only on the user previous choices, making this method robust to avoid the cold-start problem. For textual items, like articles, news and books, it is simple to use the raw text to build item profiles and user profiles.
Here we are using a very popular technique in information retrieval (search engines) named TF-IDF. This technique converts unstructured text into a vector structure, where each word is represented by a position in the vector, and the value measures how relevant a given word is for an article. As all items will be represented in the same Vector Space Model, it is to compute similarity between articles.

See this presentation (from slide 30) for more information on TF-IDF and Cosine similarity.

## Collaborative Filtering model

Collaborative Filtering (CF) has two main implementation strategies:

* Memory-based: This approach uses the memory of previous users interactions to compute users similarities based on items they've interacted (user-based approach) or compute items similarities based on the users that have interacted with them (item-based approach).
A typical example of this approach is User Neighbourhood-based CF, in which the top-N similar users (usually computed using Pearson correlation) for a user are selected and used to recommend items those similar users liked, but the current user have not interacted yet. This approach is very simple to implement, but usually do not scale well for many users. A nice Python implementation of this approach in available in Crab.
* Model-based: This approach, models are developed using different machine learning algorithms to recommend items to users. There are many model-based CF algorithms, like neural networks, bayesian networks, clustering models, and latent factor models such as Singular Value Decomposition (SVD) and, probabilistic latent semantic analysis.

### Matrix Factorization

Latent factor models compress user-item matrix into a low-dimensional representation in terms of latent factors. One advantage of using this approach is that instead of having a high dimensional matrix containing abundant number of missing values we will be dealing with a much smaller matrix in lower-dimensional space.
A reduced presentation could be utilized for either user-based or item-based neighborhood algorithms that are presented in the previous section. There are several advantages with this paradigm. It handles the sparsity of the original matrix better than memory based ones. Also comparing similarity on the resulting matrix is much more scalable especially in dealing with large sparse datasets.

Here we a use popular latent factor model named Singular Value Decomposition (SVD). There are other matrix factorization frameworks more specific to CF you might try, like surprise, mrec or python-recsys. We chose a SciPy implemenation of SVD because it is available on Kaggle kernels.
Ps. See an example of SVD on a movies dataset in this blog post.

An important decision is the number of factors to factor the user-item matrix. The higher the number of factors, the more precise is the factorization in the original matrix reconstructions. Therefore, if the model is allowed to memorize too much details of the original matrix, it may not generalize well for data it was not trained on. Reducing the number of factors increases the model generalization.

In [16]:
#Creating a sparse pivot table with users in rows and items in columns
users_items_pivot_matrix_df = interactions_train_df.pivot(index='personId', 
                                                          columns='contentId', 
                                                          values='eventStrength').fillna(0)

users_items_pivot_matrix_df.head(10)

contentId,1078,1080,1092,1131,1164,1168,1234,1404,1408,1452,...,55997,56066,56168,56169,56172,56173,56978,56979,57073,57319
personId,,,,,,,,,,,,,,,,,,,,,
22,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
64,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
84,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
125,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
132,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
176,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
205,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
221,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
289,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
users_items_pivot_matrix = users_items_pivot_matrix_df.as_matrix()
users_items_pivot_matrix[:10]

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [18]:
users_ids = list(users_items_pivot_matrix_df.index)
users_ids[:10]

[22, 64, 84, 125, 132, 176, 205, 221, 289, 413]

In [19]:
#The number of factors to factor the user-item matrix.
NUMBER_OF_FACTORS_MF = 15
#Performs matrix factorization of the original user item matrix
U, sigma, Vt = svds(users_items_pivot_matrix, k = NUMBER_OF_FACTORS_MF)

In [23]:
U.shape

(329797, 15)

In [24]:
(15, 2926)

(15, 2926)

In [26]:
sigma = np.diag(sigma)
sigma.shape

(15, 15)

After the factorization, we try to to reconstruct the original matrix by multiplying its factors. The resulting matrix is not sparse any more. It was generated predictions for items the user have not yet interaction, which we will exploit for recommendations.

In [27]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) 
all_user_predicted_ratings

array([[  2.53099409e-03,   3.14479692e-03,   3.34443973e-03, ...,
          3.26911078e-03,   1.73328149e-03,   4.52594987e-05],
       [  1.47844635e-04,   4.78430367e-04,   3.44254766e-04, ...,
          3.78200150e-04,   2.47194551e-04,  -1.05411902e-06],
       [  1.12359521e-02,   8.27874980e-03,   6.60881346e-03, ...,
          9.50092274e-03,   4.80835156e-03,   6.44766558e-04],
       ..., 
       [  8.92109903e-05,   1.65328568e-04,   6.29415080e-05, ...,
          1.11198179e-05,   3.14273336e-05,   3.60871318e-06],
       [  7.19405715e-05,   1.71454305e-04,   5.33679848e-05, ...,
          1.13446522e-04,   9.43960452e-05,   1.37496519e-06],
       [  8.60234780e-05,   1.77202688e-04,   6.80673984e-05, ...,
          2.49097185e-05,   3.30178893e-05,   1.40388537e-06]])

In [28]:
#Converting the reconstructed matrix back to a Pandas dataframe
cf_preds_df = pd.DataFrame(all_user_predicted_ratings, columns = users_items_pivot_matrix_df.columns, index=users_ids).transpose()
cf_preds_df.head(10)

,22,64,84,125,132,176,205,221,289,413,...,20087454,20087564,20091476,20092161,20092799,20094256,20094632,20095716,20099387,20102051
contentId,,,,,,,,,,,,,,,,,,,,,
1078,0.002531,0.000148,0.011236,0.000033,0.000485,0.000183,0.001256,0.001951,0.001419,0.015115,...,0.000102,0.000100,0.000072,0.000040,0.000058,0.000173,0.000214,0.000089,0.000072,0.000086
1080,0.003145,0.000478,0.008279,0.000051,0.000655,0.000285,0.002791,0.002402,0.002726,0.014347,...,0.000177,0.000157,0.000120,0.000054,0.000103,0.000206,0.000319,0.000165,0.000171,0.000177
1092,0.003344,0.000344,0.006609,0.000029,0.000327,0.000118,0.001356,0.001391,0.001156,0.007419,...,0.000075,0.000081,0.000059,0.000029,0.000046,0.000093,0.000099,0.000063,0.000053,0.000068
1131,0.001761,0.000113,0.009309,0.000025,0.000364,0.000157,0.001391,0.001299,0.001636,0.010561,...,0.000083,0.000086,0.000060,0.000033,0.000045,0.000155,0.000206,0.000068,0.000075,0.000067
1164,0.002095,0.000143,0.003131,0.000016,0.000198,0.000072,0.000540,0.000866,0.000368,0.002635,...,0.000038,0.000042,0.000031,0.000016,0.000024,0.000059,0.000066,0.000034,0.000024,0.000034
1168,0.000049,0.000008,0.000207,0.000002,0.000034,0.000016,0.000127,0.000117,0.000131,0.000628,...,0.000004,0.000006,0.000005,0.000003,0.000004,0.000007,0.000023,0.000006,0.000005,0.000003
1234,0.000601,0.000082,0.001595,0.000007,0.000095,0.000039,0.000335,0.000406,0.000271,0.001739,...,0.000025,0.000021,0.000016,0.000008,0.000013,0.000036,0.000044,0.000022,0.000020,0.000023
1404,0.000920,0.000132,0.002322,0.000013,0.000181,0.000078,0.000810,0.000676,0.000813,0.004949,...,0.000035,0.000043,0.000032,0.000017,0.000025,0.000051,0.000087,0.000034,0.000037,0.000034
1408,0.001605,0.000088,0.003561,0.000013,0.000163,0.000066,0.000686,0.000714,0.000660,0.004216,...,0.000034,0.000041,0.000029,0.000016,0.000020,0.000065,0.000074,0.000025,0.000030,0.000028


In [29]:
len(cf_preds_df.columns)

329797

In [31]:
class CFRecommender:
    
    MODEL_NAME = 'Collaborative Filtering'
    
    def __init__(self, cf_predictions_df, items_df=None):
        self.cf_predictions_df = cf_predictions_df
        self.items_df = items_df
        
    def get_model_name(self):
        return self.MODEL_NAME
        
    def recommend_items(self, user_id, items_to_ignore=[], topn=10, verbose=False):
        # Get and sort the user's predictions
        sorted_user_predictions = self.cf_predictions_df[user_id].sort_values(ascending=False) \
                                    .reset_index().rename(columns={user_id: 'recStrength'})

        # Recommend the highest predicted rating movies that the user hasn't seen yet.
        recommendations_df = sorted_user_predictions[~sorted_user_predictions['contentId'].isin(items_to_ignore)] \
                               .sort_values('recStrength', ascending = False) \
                               .head(topn)

        if verbose:
            if self.items_df is None:
                raise Exception('"items_df" is required in verbose mode')

            recommendations_df = recommendations_df.merge(self.items_df, how = 'left', 
                                                          left_on = 'contentId', 
                                                          right_on = 'contentId')[['recStrength', 'contentId', 'title', 'url', 'lang']]


        return recommendations_df

articles_df = sample_df.rename(index=str, columns = {'CUS_N_KEY_CUSTOMER': 'personId', 
                                               'PPR_N_PHY_PRODUCT_ID': 'contentId'})
cf_recommender_model = CFRecommender(cf_preds_df, articles_df)

In [ ]:
print('Evaluating Collaborative Filtering (SVD Matrix Factorization) model...')
cf_global_metrics, cf_detailed_results_df = model_evaluator.evaluate_model(cf_recommender_model)
print('\nGlobal metrics:\n%s' % cf_global_metrics)
cf_detailed_results_df.head(10)

Evaluating Collaborative Filtering (SVD Matrix Factorization) model...
